In [45]:
# coding:utf-8  
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import svm
from sklearn import metrics 

def Get_Accuracy(y_true, y_pred): #Accuracy 准确率：分类器正确分类的样本数与总样本数之比 
    accuracy = metrics.accuracy_score(y_true,y_pred)  
    return accuracy

def Get_Precision_score(y_true, y_pred): #Precision：精准率 正确被预测的正样本(TP)占所有被预测为正样本(TP+FP)的比例. 
    precision = metrics.precision_score(y_true,y_pred)  
    return precision

def Get_Recall(y_true, y_pred): #Recall 召回率 正确被预测的正样本(TP)占所有真正 正样本(TP+FN)的比例.  
    Recall = metrics.recall_score(y_true,y_pred)  
    return Recall 
 
def Get_f1_score(y_true, y_pred): #F1-score: 精确率(precision)和召回率(Recall)的调和平均数  
    f1_score = metrics.f1_score(y_true,y_pred)  
    return f1_score

#读取glove作为字典
data = pd.read_table('../GloVe/vectors.txt',header=None,encoding='gb2312',sep=' ',index_col=0)
data_dict = data.T.to_dict('list')
#读取数据源文件
#训练集
corpus = pd.read_excel('train.xls')
svm_res = list(corpus.iloc[:,4])   #输出
corpus = list(corpus.iloc[:,2])    #输入
input_size = len(svm_res)
#测试集
testset = pd.read_excel("valid.xls")  
Y_valid = list(testset.iloc[:,4])       #输出
test_valid = list(testset.iloc[:,2])    #输入
output_size = len(Y_valid)

#拼接
for i in range(len(Y_valid)):
    corpus.append(test_valid[i])

AllSen = []
len_res = []
max_len = []
# temp = corpus[9].split("\n")
for i in range(len(corpus)):
    corpus[i] = corpus[i].lower()
length = []
for i in range(len(corpus)):
    temp = []
    temp = corpus[i].replace(", ","").replace("-","").replace("("," ").replace(")"," ").replace(". ","").split(" ")
    temp = [i for i in temp if(len(str(i))!=0)]
    length.append(temp)

pca_input = []
for i in range(len(length)):
    word = []
    for j in range(613):
        if(j < len(length[i])):
            try:
                word.append(data_dict[length[i][j]])
            except:
                word.append([0 for x in range(50)])
        else:
            word.append([0 for x in range(50)])
    line = PCA(n_components=1).fit_transform(word)
    line_for_double = []
    for k in range(len(line)):
        line_for_double.append(line[k][0])
    pca_input.append(line_for_double)
#     pca_input.append(word)
    
X_train = pca_input[:input_size]
Y_train = svm_res

X_test = pca_input[input_size:]
Y_test = Y_valid

clf = svm.SVC(C=0.8, kernel='rbf', gamma=20, decision_function_shape='ovo')
# clf = svm.SVC(C=1, kernel='linear',decision_function_shape='ovo')
clf.fit(X_train, Y_train)
# pbar.update(20)
y_predict = clf.predict(X_test)  #预测

accuracy = Get_Accuracy(Y_test,y_predict)
print("SVM Accuracy_Score = %f"%accuracy) 
precision = Get_Precision_score(Y_test,y_predict)
print("SVM Precision = %f"%precision)
recall = Get_Recall(Y_test,y_predict)
print("SVM Recall = %f"%recall) 
f1_score = Get_f1_score(Y_test,y_predict)
print("SVM F1-Score  = %f"%f1_score)

/Users/futianhao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: read_table is deprecated, use read_csv instead.
/Users/futianhao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: UserWarning: DataFrame columns are not unique, some columns will be omitted.


SVM Accuracy_Score = 0.611111
SVM Precision = 0.611111
SVM Recall = 1.000000
SVM F1-Score  = 0.758621
